<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/Notebooks/extended_binomial_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extended Binomial Tree

## Introduction
The $\textbf{Extended Binomial Tree}$ is an advanced numerical method used in financial modeling, specifically for pricing derivatives like options. It builds upon the $\textit{classic binomial tree model}$ by introducing additional flexibility to handle complex financial scenarios, such as options with path dependencies, exotic features, or varying volatility.

### Key Features of the Extended Binomial Tree
- $\textbf{Multiple Branches per Node:}$
Unlike the classical binomial model, where each node splits into two outcomes (up and down), the extended model can incorporate multiple outcomes. For example, an additional "middle" state might represent no change in the underlying asset price.

- $\textbf{Time-Step Flexibility:}$
The model allows for non-uniform time steps, making it suitable for events that happen at irregular intervals, such as dividend payments.

- $\textbf{Customizable Transition Probabilities:}$
The probabilities of moving between nodes can be adjusted to reflect market conditions, such as jumps in price or shifts in volatility.

- $\textbf{Improved Precision:}$
The extended binomial tree can better approximate continuous-time models (e.g., the Black-Scholes model), especially when dealing with non-standard option structures.

## Mathematical Framework
Let $S_0$ be the initial price of the underlying asset, $T$ the maturity, $r$ the risk-free rate, $\sigma$ the volatility, and $N$ the number of time steps.

### 1. Asset Price Evolution
The price of the underlying asset at each step can evolve as:
$$
S_u = S \cdot u, \quad S_d = S \cdot d, \quad S_m = S \cdot m,
$$
where:
- $u$ is the upward movement factor ($u = e^{\sigma \sqrt{\Delta t}}$),
- $d$ is the downward movement factor ($d = e^{-\sigma \sqrt{\Delta t}}$),
- $m$ (optional) represents no movement or another intermediate state.

### 2. Transition Probabilities
The probabilities associated with each transition (\( p_u, p_d, p_m \)) are chosen to satisfy:
$$
p_u + p_d + p_m = 1,
$$
and are typically risk-neutral, ensuring consistency with the market:
$$
\mathbb{E}[S] = S \cdot e^{r \Delta t}.
$$

### 3. Pricing Backward Through the Tree
At maturity ($T$), the option's payoff depends on its type:
- $\textbf{Call Option:}$ $C(S_T) = \max(0, S_T - K)$,
- $\textbf{Put Option:}$ $P(S_T) = \max(0, K - S_T)$,
where $K$ is the strike price.

The value at earlier nodes is determined recursively:
$$
V(S_t) = e^{-r \Delta t} \big( p_u V(S_u) + p_d V(S_d) + p_m V(S_m) \big).
$$

## Advantages and Disadvantages}

### Advantages
-  $\textbf{Flexibility:}$ Can model more complex scenarios (e.g., path-dependent options, volatility jumps).
- $\textbf{Better Accuracy:}$ Improves approximation of continuous models.
- $\textbf{Handles Exotic Features:}$ Suitable for barrier options, Asian options, or those with American-style exercise.

### Disadvantages
- $\textbf{Increased Complexity:}$ Requires more computational effort compared to the classic binomial tree.
- $\textbf{Longer Calculation Time:}$ Adding branches and time flexibility increases the tree's size.

### Applications}
- $\textbf{Exotic Options:}$ Pricing complex options with path-dependence or non-standard payoffs.
- $\textbf{Local Volatility Models:}$ Accommodating scenarios where volatility varies over time or with the underlying price.
- $\textbf{Risk Management:}$ Simulating various market scenarios to evaluate risk exposure.

### Conclusion
The extended binomial tree is a powerful tool for pricing options, especially those with complex or exotic features. By introducing additional flexibility in the number of branches, time intervals, and transition probabilities, it provides a more realistic and accurate approach for modeling financial derivatives.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def extended_binomial_tree(S0, K, T, r, sigma_func, N, option_type="put"):
    """
    Extended binomial tree for option pricing with state-dependent volatility.

    Args:
        S0: Initial stock price.
        K: Strike price.
        T: Time to maturity.
        r: Risk-free rate.
        sigma_func: Function for state-dependent volatility, sigma(S, t).
        N: Number of steps in the tree.
        option_type: "call" or "put".

    Returns:
        Option price.
    """
    dt = T / N  # Time step
    u = lambda S, t: np.exp(sigma_func(S, t) * np.sqrt(dt))  # Up factor
    d = lambda S, t: 1 / u(S, t)  # Down factor

    # Initialize asset prices at maturity
    prices = np.zeros((N + 1, N + 1))
    prices[0, 0] = S0
    for i in range(1, N + 1):
        for j in range(i + 1):
            prices[j, i] = S0 * u(prices[j, i - 1], i * dt) if j == 0 else prices[j - 1, i - 1] * d(prices[j - 1, i - 1], i * dt)

    # Initialize option values at maturity
    option_values = np.maximum(0, K - prices[:, N]) if option_type == "put" else np.maximum(0, prices[:, N] - K)

    # Backward induction
    for i in reversed(range(N)):
        for j in range(i + 1):
            p = (np.exp(r * dt) - d(prices[j, i], i * dt)) / (u(prices[j, i], i * dt) - d(prices[j, i], i * dt))
            continuation_value = np.exp(-r * dt) * (p * option_values[j, i + 1] + (1 - p) * option_values[j + 1, i + 1])
            exercise_value = K - prices[j, i] if option_type == "put" else prices[j, i] - K
            option_values[j, i] = max(continuation_value, exercise_value)

    return option_values[0, 0]

# Example: American Put Option with State-Dependent Volatility
S0 = 100  # Initial stock price
K = 100   # Strike price
T = 1.0   # Time to maturity (1 year)
r = 0.05  # Risk-free rate
sigma_func = lambda S, t: 0.2 + 0.1 * np.sin(t)  # Example state-dependent volatility
N = 100   # Number of steps

price = extended_binomial_tree(S0, K, T, r, sigma_func, N, option_type="put")
print(f"American put option price: {price:.4f}")

# Visualization of Volatility
time_points = np.linspace(0, T, 100)
volatilities = [sigma_func(S0, t) for t in time_points]

plt.figure(figsize=(10, 6))
plt.plot(time_points, volatilities, label="Volatility: $\\sigma(S, t)$")
plt.xlabel("Time (t)")
plt.ylabel("Volatility ($\\sigma$)")
plt.title("State-Dependent Volatility")
plt.legend()
plt.grid()
plt.show()
\end{lstlisting}

\end{document}
